In [ ]:
import os
import cv2
from tkinter import *
from tkinter import filedialog
from shutil import copyfile
from PIL import Image, ImageTk
from ipynb.fs.full.DigitalImageProcessingUtil import *

# GUI constants
GUI_WINDOW_TITLE="RED LIGHT VIOLATIONS DETECTION SYSTEM"
GUI_WINDOW_SIZE="1920x1080"
GUI_UPLOAD_BUTTON_TEXT="UPLOAD"
GUI_WELCOME_LABEL_TEXT="Welcome to \"Determining red light violations using neural networks by analyzing traffic enforcement camera images\"!"
GUI_FILE_UPLOAD_LABEL_TEXT="Please upload a snapshot from the traffic enforcement camera and select the co-ordinates of the intended stop line."
GUI_FILE_DIALOG_LABEL_TEXT="Choose the traffic enforcement camera image"
GUI_UPLOADED_FILE_BACKUP="../file_backup/"
GUI_CANVAS_WIDTH=852
GUI_CANVAS_HEIGHT=480
GUI_DETERMINE_STOP_LINE_TEXT="DETERMINE STOP LINE"

#Stop line determination functionality constants
RESIZE_WIDTH=1280
RESIZE_HEIGHT=720
RESIZE_PIXEL_AREA_RELATION=cv2.INTER_AREA

# Defining the functions
def launchFileDialog():
    global trafficCameraImage,trafficCameraImagePath

    #Open file dialog
    filename = filedialog.askopenfile(parent=root,mode='r',title=GUI_FILE_DIALOG_LABEL_TEXT, filetypes=(("Images", "*.jpg , .png , .jpeg"), ("All Files", "*.*")))   
    if filename != None:
        
        print ('GraphicalUserInterface ---- The user selected: ', filename.name)
        
        # The uploaded file is stored in the backup location
        copyfile(filename.name, os.path.abspath(GUI_UPLOADED_FILE_BACKUP+os.path.basename(filename.name)))
        
        # A tkinter canvas is created to display the uploaded traffic enforcement camera image
        canvas = Canvas(root, width = GUI_CANVAS_WIDTH, height = GUI_CANVAS_HEIGHT)
        canvas.pack(padx=10,pady=10)
        trafficCameraImagePath=os.path.abspath(GUI_UPLOADED_FILE_BACKUP+os.path.basename(filename.name))
        trafficCameraImage = ImageTk.PhotoImage(Image.open(trafficCameraImagePath)) 
        canvas.create_image(GUI_CANVAS_WIDTH/2, GUI_CANVAS_HEIGHT/2, anchor=CENTER, image=trafficCameraImage) 
        
        return

def affirmStopLineCoordinates(event, x, y, flags, params):
    # Verifying if a left button click event was triggered or not
    if event == cv2.EVENT_LBUTTONDOWN:
        
        # Printing the x and y coordinates on the shell
        print(x, ' ', y)
        
        # Printing the x and y coordinates on the traffic enforcement camera image
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        cv2.putText(uploadedTrafficCameraImage, str(x) + ',' + str(y), (x,y), font, 1, (255, 255, 255), 2)
        
        # Displaying the image with the coordinates
        cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
        
    # Verifying if a right button click event was triggered or not
    if event==cv2.EVENT_RBUTTONDOWN:
        
        # Printing the x and y coordinates on the shell
        print(x, ' ', y)
        
        # Printing the x and y coordinates on the traffic enforcement camera image
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(uploadedTrafficCameraImage, str(b) + ',' + str(g) + ',' + str(r), (x,y), font, 1, (255, 255, 255), 2)
        
        # Displaying the image with the coordinates
        cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
        return    

def getStopLineCoordinates():
    global uploadedTrafficCameraImage
    
    # Reading the uploaded traffic enforcement camera image
    uploadedTrafficCameraImage=cv2.imread(trafficCameraImagePath, 1)
    
    # Resizing the uploaded image while maintaining aspect ratio
    uploadedTrafficCameraImage=maintainAspectRatioWhileResizing(uploadedTrafficCameraImage,RESIZE_WIDTH,RESIZE_HEIGHT,RESIZE_PIXEL_AREA_RELATION)
    
    # Displaying the resized traffic enforcement camera image
    cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
    
    # Setting the mouse call back function on the resized traffic enforcement camera image by invoking the affirmStopLineCoordinates() function
    cv2.setMouseCallback('Traffic Enforcement Camera image', affirmStopLineCoordinates)
    
    # Waiting for a key press so as to exit
    cv2.waitKey(0)
    
    # Destroying all the windows
    cv2.destroyAllWindows()
    return

def launchGUIWindow():
    global root
    # Creating a graphical user interface (GUI)
    root=Tk()
    root.title(GUI_WINDOW_TITLE)
    root.geometry(GUI_WINDOW_SIZE)

    # Welcome label in the GUI
    welcomeTextLabel=Label(root, text=GUI_WELCOME_LABEL_TEXT)
    welcomeTextLabel.pack(padx=10,pady=10)

    # File upload label in the GUI
    instructionLabel=Label(root, text=GUI_FILE_UPLOAD_LABEL_TEXT)
    instructionLabel.pack(padx=10,pady=10)


    # Traffic enforcement camera image upload button
    uploadFileButton=Button(root,text=GUI_UPLOAD_BUTTON_TEXT,command=launchFileDialog)
    uploadFileButton.pack(padx=10,pady=10)


    # Determining the stop line coordinates in the traffic enforcement camera image
    determineStopLineButton=Button(root,text=GUI_DETERMINE_STOP_LINE_TEXT,command=getStopLineCoordinates)
    determineStopLineButton.pack(padx=10,pady=10)

    root.mainloop()
    root.destroy()
    return